# Set up a Sample SQLite Database

In [1]:
import sqlite3
import pandas as pd
from twisted.conch.insults.window import cursor


def create_sample_db(db_name='sample1.db'):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    #create a table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    department TEXT NOT NULL,
    salary REAL NOT NULL)
    """)
    
    #insert sample data
    sample_data = [
        ('Alice', 'Engineering', 75000),
        ('Rob', 'HR', 50000),
        ('Charlie', 'Finance', 65000),
        ('David', 'Engineering', 80000),
        ('Arthur', 'Marketing', 60000),
        ('Ginnie', "HR", 50000)
    ]
    
    cursor.executemany("INSERT INTO employees (name, department, salary) VALUES (?, ?, ?)", sample_data)
    conn.commit()
    conn.close()
    print(f"Database '{db_name}' created and populated with sample data.")
    return db_name

#Test
db_path = create_sample_db()

Database 'sample1.db' created and populated with sample data.


# Connect to Database

In [5]:
def connect_to_db(db_name):
    try:
        conn = sqlite3.connect(db_name)
        print(f"Connected to {db_name}")
        conn.close()
        return conn
    except sqlite3.Error as e:
        print(f'Error connecting to database: {e}')
        return None
    
#Test
connection = connect_to_db(db_path)

Connected to sample1.db


# Execute SQL Queries

In [3]:
def execute_query(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        
        #Fetch column names, data
        columns = [desc[0] for desc in cursor.description]
        data = cursor.fetchall()
        
        #return as pandas DF
        return pd.DataFrame(data, columns=columns)
    except sqlite3.Error as e:
        print(f'Error executing query: {e}')
        return None
    
#Test
test_query = 'SELECT * FROM employees'
result_df = execute_query(connection, test_query)

#Display results
if result_df is not None:
    print(result_df)

   id     name   department   salary
0   1    Alice  Engineering  75000.0
1   2      Rob           HR  50000.0
2   3  Charlie      Finance  65000.0
3   4    David  Engineering  80000.0
4   5   Arthur    Marketing  60000.0
5   6   Ginnie           HR  50000.0


# Create an interface with ipywidgets

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def interactive_interface(connection):
    query_input = widgets.Textarea(
        value="SELECT * FROM employees",
        placeholder="Type your SQL query here...",
        description="Query:",
        layout=widgets.Layout(width="80%", height='100px')
    )
    execute_button = widgets.Button(description="Execute Query", button_style='success')
    output_area = widgets.Output()
    
    #Callback func
    def on_button_clicked(b):
        with output_area:
            clear_output()
            query = query_input.value
            result_df = execute_query(connection, query)
            if result_df is not None:
                display(result_df)
            else:
                print('No results or invalid query.')
                
    execute_button.on_click(on_button_clicked)
    
    #Display widgets
    display(query_input, execute_button, output_area)
    
#Test
interactive_interface(connection)

Textarea(value='SELECT * FROM employees', description='Query:', layout=Layout(height='100px', width='80%'), pl…

Button(button_style='success', description='Execute Query', style=ButtonStyle())

Output()

# Error Handling

In [5]:
def execute_query(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        
        #Fetch column names, data
        columns = [desc[0] for desc in cursor.description] if cursor.description else None
        data = cursor.fetchall()
        
        #return as pandas DF
        return pd.DataFrame(data, columns=columns) if columns else pd.DataFrame()
    except sqlite3.Error as e:
        print(f'SQL Error: {e}')
        return None

# Database Schema

In [7]:
def display_schema(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
        tables = cursor.fetchall()
        
        print('Database Schema')
        for table in tables:
            table_name = table[0]
            print(f'\nTable: {table_name}')
            cursor.execute(f"PRAGMA TABLE_INFO({table_name})")
            columns = cursor.fetchall()
            for column in columns:
                print(f' - {column[1]} ({column[2]})')
    except sqlite3.Error as e:
        print(f'Error retrieving schema: {e}')
        
#Test 
connection = sqlite3.connect('sample.db')
display_schema(connection)

Database Schema

Table: employees
 - id 0 (INTEGER)
 - name 1 (TEXT)
 - department 2 (TEXT)
 - salary 3 (REAL)

Table: sqlite_sequence
 - name 0 ()
 - seq 1 ()


# Export Query Results

def export_query(result_df, file_name='query_results.csv'):
    try:
        result_df.to_csv(file_name, index=False)
        print(f'Query results exported to "{file_name}".')
    except Exception as e:
        print(f'Error saving query results: {e}')
        
        
#TEst
test_query = 'SELECT * FROM employees'
result_df = execute_query(connection, test_query)
if result_df is not None:
    export_query(result_df)

# Upload DBs

def upload_db():
    upload_widget = widgets.FileUpload(
        accept='.db',
        multiple=False
    )
    
    def handle_upload(change):
        if upload_widget.value:
            uploaded_file = next(iter(upload_widget.value.values()))
            db_name = uploaded_file['name']
            with open(db_name, 'wb') as f:
                f.write(uploaded_file['content'])
            print(f'Database "{db_name}" uploaded successfully!')
    
    upload_widget.observe(handle_upload, names='value')
    display(upload_widget)
    
    
#test
upload_db()

# All together

import sqlite3
import pandas as pd
from twisted.conch.insults.window import cursor
import ipywidgets as widgets
from IPython.display import display, clear_output

def query_interface():
    upload_widget = widgets.FileUpload(accept='.db', multiple=False)
    query_input = widgets.Textarea(placeholder="Type your SQL query here...", layout=widgets.Layout(width="100%"))
    submit_query_button = widgets.Button(description="Run Query", button_style='success')
    export_button = widgets.Button(description="Export Query", button_style='primary')
    schema_output = widgets.Output()
    query_output = widgets.Output()
    export_output = widgets.Output()
    
    #DB connection placeholder
    db_connection =  [None]
    result_df = [None]
    
    def upload_db(change):
        with schema_output:
            clear_output()
            try:
                if not upload_widget.value:
                    print('No file uploaded. Please upload a .db file.')
                    return 
                
                uploaded_file = next(iter(upload_widget.value.values()))
                db_name = uploaded_file['name']
                
                with open(db_name, 'wb') as f:
                    f.write(uploaded_file['content'])
                
                conn = sqlite3.connect(db_name)
                db_connection[0] = conn
                print(f"Database '{db_name}' uploaded successfully!")
                
                display_schema(db_connection[0])
            except sqlite3.Error as e:
                print(f'SQLite Error: {e}')
            except Exception as e:
                print(f'Error uploading database: {e}')
                
            
    def display_schema(connection):
        try:
            cursor = connection.cursor()
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
            tables = cursor.fetchall()
            if not tables:
                print('No tables found in the database.')
                return 
            
            print('Database Schema:')
            
            for table in tables:
                table_name = table[0]
                print(f'\nTable: {table_name}')
                cursor.execute(f"PRAGMA TABLE_INFO({table_name})")
                columns = cursor.fetchall()
                for column in columns:
                    print(f' - {column[1]} ({column[2]})')
        except sqlite3.Error as e:
            print(f'Error retrieving schema: {e}')
            
    
    def validate_query(query):
        prohibited_commands = ["DROP", "DELETE", "UPDATE"]
        for cmd in prohibited_commands:
            if cmd in query.upper():
                print(f"Query contains prohibited command: {cmd}")
                return False
        return True
    
    
    def execute_query(query):
        try:
            if db_connection[0] is None:
                print("No database connected. Please connect to the database.")
                return None
            if not validate_query(query):
                return None
            
            cursor = db_connection[0].cursor()
            cursor.execute(query)
            if query.strip().upper().startswith("SELECT"):
                rows = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]
                return pd.DataFrame(rows, columns=columns)
            else:
                db_connection[0].commit()
                print(f"Query executed successfully!")
        except sqlite3.Error as e:
            print(f'Error executing query: {e}')
        return None
    
    
    def run_query(button):
        with query_output:
            clear_output()
            query = query_input.value.strip()
            if not query:
                print('Please enter a SQL query.')
                return 
            
            df = execute_query(query)
            if df is not None:
                print("Query Results:")
                display(df)
                result_df[0] = df
            else: 
                result_df[0] = None
                
    def export_results(button):
        with export_output:
            clear_output()
            if result_df[0] is not None:
                result_df[0].to_csv('query_results.csv', index=False)
                print(f'Query results exported to "query_results.csv".')
            else:
                print('No query results to export.')
                
    #Widgets
    upload_widget.observe(upload_db, names='value')
    submit_query_button.on_click(run_query)
    export_button.on_click(export_results)
    
    #Display widgets
    display(widgets.HTML("<h3>SQLite Interactive Query Interface</h3>"))
    display(widgets.VBox([
        widgets.HTML("<b>Step 1: Upload a SQLite database file (.db)</b>"),
        upload_widget,
        schema_output,
        widgets.HTML("<hr><b>Step 2: Enter your SQL query</b>"),
        query_input,
        submit_query_button,
        query_output,
        widgets.HTML("<hr><b>Step 3: Export Query Results (Optional)</b>"),
        export_button,
        export_output
    ]))
    

#Run!
query_interface()